In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Read csv data
purchases=pd.read_csv("purchases.csv",encoding="cp1252")

In [3]:
# Data snapshot
purchases.head()

,qty,item_i,gst_i,purchase_d
0,4.0,122464,2639949,23/08/2016_00:00:00
1,1.0,99091,4935278,16/08/2015_00:00:00
2,1.0,119976,3479638,18/12/2015_00:00:00
3,1.0,22501,257693,06/08/2016_00:00:00
4,1.0,23785,1912070,06/01/2016_00:00:00


In [4]:
# Summarize the number of rows and columns
print(purchases.shape)

(37796, 4)


# Data Preprocessing

In [5]:
# Count of missing observations
purchases.isnull().sum()

qty            0
item_i        98
gst_i          0
purchase_d     0
dtype: int64

In [6]:
# Drop rows with missing values
purchases.dropna(inplace=True)

In [7]:
# Check for qty distribution
purchases.qty.describe()

count    37698.000000
mean         1.328630
std          1.136038
min         -1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max         63.000000
Name: qty, dtype: float64

In [9]:
# We see there exists entries with Quantity purchased <0 which is not possible
# Number of such rows
len(purchases[purchases.qty<=0])

98

In [10]:
# Rows where Item id has special characters
print (purchases[purchases.item_i.str.contains('\?')==True])

       qty   item_i    gst_i           purchase_d
1833   1.0  Š—È?Š—È  1461496  27/11/2015_00:00:00
24965  1.0        ?  2446816  23/03/2016_00:00:00


In [11]:
# Columns where Guest id has special characters
print (purchases[purchases.gst_i.str.contains('\?')==True])

       qty  item_i    gst_i           purchase_d
6619   1.0  110411        ?  22/10/2016_00:00:00
11893  1.0   58320  Š—È?Š—È  04/04/2016_00:00:00


In [12]:
# Exclude rows with qty<=0 & Guest id/Item id containing special characters
clean_purchases=purchases[(purchases.gst_i.str.contains('\?')==False) & (purchases.item_i.str.contains('\?')==False) 
           & (purchases.qty>0) ]
clean_purchases.shape

(37596, 4)

In [18]:
# Check whether a guest purchases an item multiple times on different days
clean_purchases.groupby(['gst_i','item_i']).size().describe()

count    37596.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
dtype: float64

In [15]:
# Since a guest purchases an item just once, there is no use of purchase date column and we can drop it.

# Final Clean Data for Exploratory Analysis
exploratory_pur_data=clean_purchases.drop('purchase_d', axis=1)

In [356]:
# Count number of guests and items in the dataset

print(str(exploratory_pur_data.gst_i.unique().shape[0]) + ' guests')
print(str(exploratory_pur_data.item_i.unique().shape[0]) + ' items')

1000 guests
17403 items


# Exploratory Analysis

In [35]:
# Distribution of Items purchased by guests

exploratory_pur_data.groupby(['gst_i']).agg({'item_i': 'count'}).reset_index().describe()

#On average a guest purchased 38 items

,item_i
count,1000.00000
mean,37.59600
std,32.50094
min,1.00000
25%,14.00000
50%,28.00000
75%,50.25000
max,281.00000


In [34]:
#Median # of items purchased is 28

exploratory_pur_data.groupby(['gst_i']).agg({'item_i': 'count'}).reset_index().median()

gst_i     2578025.0
item_i         28.0
dtype: float64

In [26]:
# Identify top 10 guests who purchased maximum items

print(exploratory_pur_data.groupby(['gst_i']).agg({'item_i': 'count'}).reset_index().
      sort_values(['item_i', 'gst_i'], ascending = [0,1]).head(10))

       gst_i  item_i
212  1904015     281
785   466727     191
342  2563664     174
356     2629     168
361  2639949     164
326  2482638     154
721  4414710     154
531  3384086     153
974    87626     153
549  3500517     149


In [30]:
# Drop items from the dataset which has been purchased maximum 3 times

purchases_drop_uncommon_items = exploratory_pur_data.groupby('item_i').filter(lambda x: len(x) >3)

print('----------AFTER DROPPING ITEMS WHICH HAVE BEEN PURCHASED ONCE ---------------')
print(str(purchases_drop_uncommon_items.gst_i.unique().shape[0]) + ' guests')
print(str(purchases_drop_uncommon_items.item_i.unique().shape[0]) + ' items')

----------AFTER DROPPING ITEMS WHICH HAVE BEEN PURCHASED ONCE ---------------
955 guests
2402 items


In [32]:
# How many items have been purchased by the guests on average who have been dropped?

# List of users who have been dropped
dropped_users = list(set(exploratory_pur_data.gst_i.unique()) - set(purchases_drop_uncommon_items.gst_i.unique()))

# All items purchased by the dropped users
Dropped_rows=exploratory_pur_data[exploratory_pur_data['gst_i'].isin(dropped_users)]

# Distribution of items purchased by these users
Dropped_rows.groupby(['gst_i']).agg({'item_i': 'count'}).reset_index().describe()

,item_i
count,45.000000
mean,5.933333
std,3.834058
min,1.000000
25%,3.000000
50%,5.000000
75%,8.000000
max,17.000000


In [37]:
#Median # of items purchased by these users is 5
Dropped_rows.groupby(['gst_i']).agg({'item_i': 'count'}).reset_index().median()

gst_i     2853587.0
item_i          5.0
dtype: float64